In [1]:
%load_ext autoreload
%autoreload 2
import torch 
from torch import nn
from omegaconf import OmegaConf
import timm
from ml_utilities.output_loader.result_loader import JobResult
from ml_utilities.data.datasetgenerator import DatasetGenerator
from ml_utilities.data.data_utils import show_image_grid
from torch.utils import data
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tflearning.sample_difficulty import prediction_depth as pdepth

In [2]:
job_cf10resnet = JobResult('/system/user/beck/pwbeck/projects/regularization/tflearning/outputs/IA-A-cifar10-17.2.2-resnet-B--230204_215821')
job_cf10resnet 

JobResult(/system/user/publicwork/beck/projects/regularization/tflearning/outputs/IA-A-cifar10-17.2.2-resnet-B--230204_215821)

In [3]:
print(OmegaConf.to_yaml(job_cf10resnet.config.config.data))

dataset: cifar10
dataset_kwargs:
  data_root_path: /system/user/beck/pwbeck/data
dataset_split:
  train_val_split: 0.9
train_split_transforms:
  image_transforms:
  - RandomHorizontalFlip
  - RandomCrop:
      size: 32
      padding: 4
  tensor_transforms: null
  joint_tensor_transforms: null
  enable_transforms: true



In [4]:
# create dataset cf10 train split
data_cfg = """
data:
  dataset: cifar10
  dataset_kwargs:
    data_root_path: /system/user/beck/pwbeck/data
  dataset_split:
    train_val_split: 0.9
"""
data_cfg = OmegaConf.create(data_cfg)
cf10_ds_gen = DatasetGenerator(**data_cfg.data)
cf10_ds_gen.generate_dataset()
cf10_train = cf10_ds_gen.train_split
cf10_val = cf10_ds_gen.val_split

Files already downloaded and verified


In [5]:
cf10_trainloader = data.DataLoader(cf10_train, batch_size=128, shuffle=False)
cf10_valloader = data.DataLoader(cf10_val, batch_size=128, shuffle=False)

In [6]:
x, y = next(iter(cf10_trainloader))

In [7]:
# show_image_grid(x, normalize=True)

In [8]:
myresnet = job_cf10resnet.get_model_idx(-1)

In [9]:
layer_names = pdepth.find_layer_names(myresnet, 'act2')
layer_names.append('resnet.0.2') # add the last layer (activation) of the first block

In [10]:
lfext = pdepth.LayerFeatureExtractor(myresnet, layer_names)
lfext = lfext.to('cuda')

In [11]:
next(iter(lfext.parameters())).device

device(type='cuda', index=0)

In [12]:
x = x.to('cuda')
pred, feats = lfext(x)

In [13]:
feats.keys()

dict_keys(['resnet.0.2', 'resnet.1.0.0.act2', 'resnet.1.0.1.act2', 'resnet.1.0.2.act2', 'resnet.1.1.0.act2', 'resnet.1.1.1.act2', 'resnet.1.1.2.act2', 'resnet.1.2.0.act2', 'resnet.1.2.1.act2', 'resnet.1.2.2.act2', 'softmax_output'])

In [14]:
for k, v in feats.items():
    print(k, v.shape)

resnet.0.2 torch.Size([128, 16, 32, 32])
resnet.1.0.0.act2 torch.Size([128, 16, 32, 32])
resnet.1.0.1.act2 torch.Size([128, 16, 32, 32])
resnet.1.0.2.act2 torch.Size([128, 16, 32, 32])
resnet.1.1.0.act2 torch.Size([128, 32, 16, 16])
resnet.1.1.1.act2 torch.Size([128, 32, 16, 16])
resnet.1.1.2.act2 torch.Size([128, 32, 16, 16])
resnet.1.2.0.act2 torch.Size([128, 64, 8, 8])
resnet.1.2.1.act2 torch.Size([128, 64, 8, 8])
resnet.1.2.2.act2 torch.Size([128, 64, 8, 8])
softmax_output torch.Size([128, 10])


In [15]:
timm_resnet = timm.create_model('resnet18', pretrained=False, num_classes=10)

In [16]:
for name, module in timm_resnet.named_modules():
    print(name)


conv1
bn1
act1
maxpool
layer1
layer1.0
layer1.0.conv1
layer1.0.bn1
layer1.0.drop_block
layer1.0.act1
layer1.0.aa
layer1.0.conv2
layer1.0.bn2
layer1.0.act2
layer1.1
layer1.1.conv1
layer1.1.bn1
layer1.1.drop_block
layer1.1.act1
layer1.1.aa
layer1.1.conv2
layer1.1.bn2
layer1.1.act2
layer2
layer2.0
layer2.0.conv1
layer2.0.bn1
layer2.0.drop_block
layer2.0.act1
layer2.0.aa
layer2.0.conv2
layer2.0.bn2
layer2.0.act2
layer2.0.downsample
layer2.0.downsample.0
layer2.0.downsample.1
layer2.1
layer2.1.conv1
layer2.1.bn1
layer2.1.drop_block
layer2.1.act1
layer2.1.aa
layer2.1.conv2
layer2.1.bn2
layer2.1.act2
layer3
layer3.0
layer3.0.conv1
layer3.0.bn1
layer3.0.drop_block
layer3.0.act1
layer3.0.aa
layer3.0.conv2
layer3.0.bn2
layer3.0.act2
layer3.0.downsample
layer3.0.downsample.0
layer3.0.downsample.1
layer3.1
layer3.1.conv1
layer3.1.bn1
layer3.1.drop_block
layer3.1.act1
layer3.1.aa
layer3.1.conv2
layer3.1.bn2
layer3.1.act2
layer4
layer4.0
layer4.0.conv1
layer4.0.bn1
layer4.0.drop_block
layer4.0.act1

In [17]:
pred_depth = pdepth.PredictionDepth(myresnet, layer_names, cf10_valloader)

In [18]:
pred_depth._extract_features()

AttributeError: 'list' object has no attribute 'items'

In [38]:
len(pred_depth.feature_batches)

40

In [39]:
len(cf10_valloader)

40

In [41]:
type(pred_depth.label_batches)

list